In [8]:
using Gen
using GenArrow
using Serialization
using BenchmarkTools

In [9]:
@gen function model()
    x ~ mvnormal([0, 0], [1 0; 0 1])
    if (b ~ bernoulli(0.5))
        y ~ categorical([0.25, 0.25, 0.25, 0.25])
        {:a => 1} ~ bernoulli(0.5)
    else
        z ~ exponential(2)
        {:c => 1} ~ bernoulli(0.5)
    end
    return 1
end
@gen function foo(x1::Float64, x2::Float64)
    y = @trace(normal(x1 + x2, 1.0), :z)
    return y
end
@gen function zoobar(t::Int, y_prev::Bool, z1::Float64, z2::Float64)
    y = @trace(bernoulli(y_prev ? z1 : z2), :y)
    return y
end

bar = Map(foo)
zoo = Unfold(zoobar)


Unfold{Any, Gen.DynamicDSLTrace}(DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Int64, Bool, Float64, Float64], false, Union{Nothing, Some{Any}}[nothing, nothing, nothing, nothing], var"##zoobar#317", Bool[0, 0, 0, 0], false))

In [60]:
tr_old, w_old = generate(model, ())
(trace, _) = generate(bar, ([0.0, 0.5], [0.5, 1.0]))
(brace, _) = generate(zoo, (5, false, 0.05, 0.95))

(Gen.VectorTrace{Gen.UnfoldType, Any, Gen.DynamicDSLTrace}(Unfold{Any, Gen.DynamicDSLTrace}(DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Int64, Bool, Float64, Float64], false, Union{Nothing, Some{Any}}[nothing, nothing, nothing, nothing], var"##zoobar#317", Bool[0, 0, 0, 0], false)), Gen.DynamicDSLTrace[Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Int64, Bool, Float64, Float64], false, Union{Nothing, Some{Any}}[nothing, nothing, nothing, nothing], var"##zoobar#317", Bool[0, 0, 0, 0], false), Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}(:y => Gen.ChoiceOrCallRecord{Bool}(false, -2.99573227355399, NaN, true)), Dict{Any, Trie{Any, Gen.ChoiceOrCallRecord}}()), false, -2.99573227355399, 0.0, (1, false, 0.05, 0.95), false), Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Int64, Bool, Float64, Float64], fals

In [108]:
function bench()
    io = GenArrow.serialize(trace);
end
@benchmark bench()

LoadError: LoadError: UndefVarError: @benchmark not defined
in expression starting at /Users/ian/Documents/probcomp/GenArrow.jl/test/unit/s.ipynb:4

In [105]:
seekstart(io)
GenArrow.deserialize(bar, io)

2 2 2 2 0.0
Deserialize type: Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}
Deserialize type: Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}
